#1. Data preparation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchtext
import random
import tarfile
import requests
import tarfile
from tqdm import tqdm

In [ ]:
BATCHSIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download and extract data
def iwslt15(train_test):
    url ='https://github.com/stefan-it/nmt-en-vi/raw/master/data/'
    r = requests.get(url + train_test + '-en-vi.tgz')
    filename = train_test + '-en-vi.tgz'
    with open(filename, 'wb') as f:
        f.write(r.content)
        tarfile.open(filename, "r:gz").extractall("iwslt15")

# Load data
iwslt15('train')
iwslt15('test-2013')

In [ ]:
import os

# List all files in the current directory
files = os.listdir()

# Print the list of files
for file in files:
    print(file)

.config
test-2013-en-vi.tgz
train-en-vi.tgz
iwslt15
sample_data


In [ ]:
import tarfile
import os

# Open the .tgz file
tar = tarfile.open('train-en-vi.tgz', 'r:gz')

# Extract all files into the current directory
tar.extractall()

# Close the .tgz file
tar.close()

In [ ]:
!pip install bpemb

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

# Đọc file 'train.en' vào DataFrame
df_train_en = pd.read_csv('train.en', delimiter="\t", header=None)
rtpe =[121099,121582]
print("Before replacement:")
values_before_replacement = []
for row in rtpe:
    if row < len(df_train_en):
        values_before_replacement.append((row, df_train_en.iloc[row, 0]))
        print(f"Row {row}: {df_train_en.iloc[row, 0]}")
# Thay thế các giá trị không hợp lệ bằng NaN trong DataFrame 'df_train_vi'
for row in rtpe:
    if row < len(df_train_en):
        df_train_en.at[row, 0] = np.nan

df_train_en.dropna(inplace=True)
# Đọc file 'train.vi' vào DataFrame
df_train_vi = pd.read_csv('train.vi', delimiter="\t", header=None)

# Danh sách các dòng cần thay thế bằng NaN, đã trừ đi 2 để đúng với DataFrame
rows_to_replace = [469, 8695, 9762, 10707, 21738, 26408, 29494, 38600, 41017, 48825, 50894, 51586,
                   54158, 56297, 57140, 57746, 58330, 66260, 68755, 73578, 76896, 77188, 79669,
                   87612, 88637, 89931, 94060, 104125, 104677, 107006, 112823, 115182, 116898,
                   123074, 124916, 125209, 128025,130121,130770]
# In ra các giá trị của các ô trước khi thay thế
print("Before replacement:")
values_before_replacement = []
for row in rows_to_replace:
    if row < len(df_train_vi):
        values_before_replacement.append((row, df_train_vi.iloc[row, 0]))
        print(f"Row {row}: {df_train_vi.iloc[row, 0]}")

# Thay thế các giá trị không hợp lệ bằng NaN trong DataFrame 'df_train_vi'
for row in rows_to_replace:
    if row < len(df_train_vi):
        df_train_vi.at[row, 0] = np.nan

# In ra các giá trị đã thay thế để kiểm tra xem các giá trị có đúng là NaN không
print("\nReplaced values (should be NaN):")
for row in rows_to_replace:
    if row < len(df_train_vi):
        print(f"Row {row}: {df_train_vi.iloc[row, 0]}")
df_train_vi.dropna(inplace=True)

# Đổi tên các cột
df_train_en.columns = ['en']
df_train_vi.columns = ['vi']

# Nối hai DataFrame lại với nhau
df_train = pd.concat([df_train_vi.reset_index(drop=True), df_train_en.reset_index(drop=True)], axis=1)

# Chuyển cột 'en' sang kiểu string
df_train['en'] = df_train['en'].astype(str)


Before replacement:
Row 121099: .
Row 121582: .
Before replacement:
Row 469: ̣
Row 8695: Khán giả vỗ tay
Row 9762: Vỗ tay .
Row 10707: vỗ tay
Row 21738: Vỗ tay
Row 26408: vỗ tay
Row 29494: Vỗ tay .
Row 38600: &quot; tán thưởng &quot;
Row 41017: Vỗ tay
Row 48825: Vỗ tay
Row 50894: .
Row 51586: .
Row 54158: Vỗ tay
Row 56297: .
Row 57140: Vỗ tay
Row 57746: .
Row 58330: Vỗ tay
Row 66260: ̣
Row 68755: Cười
Row 73578: Vỗ tay
Row 76896: Vỗ tay
Row 77188: .
Row 79669: .
Row 87612: .
Row 88637: .
Row 89931: Vỗ tay
Row 94060: Vỗ tay .
Row 104125: Vỗ tay
Row 104677: Vỗ tay
Row 107006: Vỗ tay
Row 112823: vỗ tay
Row 115182: Vỗ tay
Row 116898: Vỗ tay .
Row 123074: Cám ơn
Row 124916: .
Row 125209: .
Row 128025: .
Row 130121: có những giấc mơ đẹp .
Row 130770: .

Replaced values (should be NaN):
Row 469: nan
Row 8695: nan
Row 9762: nan
Row 10707: nan
Row 21738: nan
Row 26408: nan
Row 29494: nan
Row 38600: nan
Row 41017: nan
Row 48825: nan
Row 50894: nan
Row 51586: nan
Row 54158: nan
Row 56297: nan
Row

In [ ]:
from bpemb import BPEmb

bpemb_vi = BPEmb(lang="vi", vs=50000, dim=200)
bpemb_en = BPEmb(lang="en", vs=50000, dim=200)

downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs50000.model


100%|██████████| 1041864/1041864 [00:00<00:00, 1470452.23B/s]


downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs50000.d200.w2v.bin.tar.gz


100%|██████████| 37483223/37483223 [00:04<00:00, 9275250.40B/s] 


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs50000.model


100%|██████████| 1100587/1100587 [00:00<00:00, 1503888.07B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs50000.d200.w2v.bin.tar.gz


100%|██████████| 37527847/37527847 [00:03<00:00, 10339361.89B/s]


In [ ]:
def tokenizer(text, lang):
  if lang == "vi":
    text_pp = " ".join(bpemb_vi.encode(text))
  else:
    text_pp = " ".join(bpemb_en.encode(text))

  return text_pp

In [ ]:
def token_len(text):
    return len(text.replace("▁", "").split(" "))

In [ ]:
df_train["vi_token"] = df_train["vi"].apply(lambda x: tokenizer(x, "vi"))
df_train["en_token"] = df_train["en"].apply(lambda x: tokenizer(x, "en"))

# In ra số lượng giá trị không phải là NaN trong mỗi cột
print("\nCount of non-NaN values in each column:")
print(df_train.count())



Count of non-NaN values in each column:
vi          133166
en          133166
vi_token    133166
en_token    133166
dtype: int64


In [ ]:
tokenizer(df_train["vi"][0], "vi")

'▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ▁hậu'

In [ ]:
df_train = df_train.head(133000)
df_train

,vi,en,vi_token,en_token
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike ▁: ▁the ▁science ▁behind ▁a ▁cli...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...
...,...,...,...,...
132995,"Cái cô cần là sự cảm thông và lòng trắc ẩn , v...","You needed my empathy and compassion , and abo...",▁cái ▁cô ▁cần ▁là ▁sự ▁cảm ▁thông ▁và ▁lòng ▁t...,"▁you ▁needed ▁my ▁empathy ▁and ▁compassion ▁, ..."
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros ▁: ▁làm ▁sao ▁để ▁bớt ▁nghèo...,▁paul ▁ph ol eros ▁: ▁how ▁to ▁reduce ▁poverty...


In [ ]:
import tarfile
import os

# Open the .tgz file
tar = tarfile.open('test-2013-en-vi.tgz', 'r:gz')

# Extract all files into the current directory
tar.extractall()

# Close the .tgz file
tar.close()


In [ ]:
import pandas as pd
df_test_en = pd.read_csv('tst2013.en', delimiter = "\t", header=None)

df_test_vi = pd.read_csv('tst2013.vi', delimiter = "\t", header=None)

import pandas as pd
import re

# Rename the columns
df_test_en.columns = ['en']
df_test_vi.columns = ['vi']

# Concatenate the dataframes
df_test = pd.concat([df_test_vi, df_test_en], axis=1)
# Convert 'en' column to string and then create the 'en_token' column
df_test['en'] = df_test['en'].astype(str)


df_test["vi_token"] = df_test["vi"].apply(lambda x: tokenizer(x, "vi"))
df_test["en_token"] = df_test["en"].apply(lambda x: tokenizer(x, "en"))

In [ ]:
df_test

,vi,en,vi_token,en_token
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m..."
...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...


In [ ]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
from sacremoses import MosesPunctNormalizer

mpn_vi = MosesPunctNormalizer(lang = "vi")
mpn_en = MosesPunctNormalizer(lang = "en")

def punc_norm(text, lang):
  if lang == "vi":
    return mpn_vi.normalize(text)

  return mpn_en.normalize(text)

In [ ]:
df_train["vi_punc"] = df_train["vi"].apply(lambda x: punc_norm(x, "vi"))
df_train["en_punc"] = df_train["en"].apply(lambda x: punc_norm(x, "en"))

df_train

,vi,en,vi_token,en_token,vi_punc,en_punc
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike ▁: ▁the ▁science ▁behind ▁a ▁cli...,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike: The science behind a climate head...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach...","Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...
...,...,...,...,...,...,...
132995,"Cái cô cần là sự cảm thông và lòng trắc ẩn , v...","You needed my empathy and compassion , and abo...",▁cái ▁cô ▁cần ▁là ▁sự ▁cảm ▁thông ▁và ▁lòng ▁t...,"▁you ▁needed ▁my ▁empathy ▁and ▁compassion ▁, ...","Cái cô cần là sự cảm thông và lòng trắc ẩn , v...","You needed my empathy and compassion , and abo..."
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁...","Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,...","Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros ▁: ▁làm ▁sao ▁để ▁bớt ▁nghèo...,▁paul ▁ph ol eros ▁: ▁how ▁to ▁reduce ▁poverty...,Paul Pholeros: Làm sao để bớt nghèo khổ ? Hãy ...,Paul Pholeros: How to reduce poverty ? Fix homes


In [ ]:
df_test["vi_punc"] = df_test["vi"].apply(lambda x: punc_norm(x, "vi"))
df_test["en_punc"] = df_test["en"].apply(lambda x: punc_norm(x, "en"))

df_test

,vi,en,vi_token,en_token,vi_punc,en_punc
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun...","Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s...","Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m...","Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu..."
...,...,...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁...","Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...


In [ ]:
df_train["vi_token"] = df_train["vi_punc"].apply(lambda x: tokenizer(x, "vi"))
df_train["en_token"] = df_train["en_punc"].apply(lambda x: tokenizer(x, "en"))

df_train

,vi,en,vi_token,en_token,vi_punc,en_punc
0,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike : The science behind a climate hea...,▁khoa ▁học ▁đằng ▁sau ▁một ▁tiêu ▁đề ▁về ▁khí ...,▁rachel ▁pike : ▁the ▁science ▁behind ▁a ▁clim...,Khoa học đằng sau một tiêu đề về khí hậu,Rachel Pike: The science behind a climate head...
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike...","▁trong ▁0 ▁phút ▁, ▁chuyên ▁gia ▁hoá ▁học ▁khí...","▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rach...","Trong 4 phút , chuyên gia hoá học khí quyển Ra...","In 4 minutes , atmospheric chemist Rachel Pike..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...,▁tôi ▁muốn ▁cho ▁các ▁bạn ▁biết ▁về ▁sự ▁to ▁l...,▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁toda...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,I &apos;d like to talk to you today about the ...
3,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...,▁có ▁những ▁dòng ▁trông ▁như ▁thế ▁này ▁khi ▁b...,▁headlines ▁that ▁look ▁like ▁this ▁when ▁they...,Có những dòng trông như thế này khi bàn về biế...,Headlines that look like this when they have t...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...,▁cả ▁hai ▁đều ▁là ▁một ▁nhánh ▁của ▁cùng ▁một ...,▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same...,Cả hai đều là một nhánh của cùng một lĩnh vực ...,They are both two branches of the same field o...
...,...,...,...,...,...,...
132995,"Cái cô cần là sự cảm thông và lòng trắc ẩn , v...","You needed my empathy and compassion , and abo...",▁cái ▁cô ▁cần ▁là ▁sự ▁cảm ▁thông ▁và ▁lòng ▁t...,"▁you ▁needed ▁my ▁empathy ▁and ▁compassion ▁, ...","Cái cô cần là sự cảm thông và lòng trắc ẩn , v...","You needed my empathy and compassion , and abo..."
132996,"Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was...",▁mà ▁chính ▁hệ ▁thống ▁y ▁tế ▁mà ▁tôi ▁góp ▁ph...,"▁maybe ▁the ▁system ▁, ▁of ▁which ▁i ▁was ▁a ▁...","Mà chính hệ thống y tế mà tôi góp phần , đã là...","Maybe the system , of which I was a part , was..."
132997,"Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you...",▁nếu ▁giờ ▁đây ▁cô ▁đang ▁xem ▁chương ▁trình ▁...,"▁if ▁you ▁& ap os ; re ▁watching ▁this ▁now ▁,...","Nếu giờ đây cô đang xem chương trình này , tôi...","If you &apos;re watching this now , I hope you..."
132998,Paul Pholeros : Làm sao để bớt nghèo khổ ? Hãy...,Paul Pholeros : How to reduce poverty ? Fix homes,▁paul ▁ph ol eros : ▁làm ▁sao ▁để ▁bớt ▁nghèo ...,▁paul ▁ph ol eros : ▁how ▁to ▁reduce ▁poverty ...,Paul Pholeros: Làm sao để bớt nghèo khổ ? Hãy ...,Paul Pholeros: How to reduce poverty ? Fix homes


In [ ]:
df_test["vi_token"] = df_test["vi_punc"].apply(lambda x: tokenizer(x, "vi"))
df_test["en_token"] = df_test["en_punc"].apply(lambda x: tokenizer(x, "en"))

df_test

,vi,en,vi_token,en_token,vi_punc,en_punc
0,"Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t...","▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc t...","▁when ▁i ▁was ▁little ▁, ▁i ▁thought ▁my ▁coun...","Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ...","When I was little , I thought my country was t..."
1,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .,▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.,▁and ▁i ▁was ▁very ▁proud ▁.,Tôi đã rất tự hào về đất nước tôi .,And I was very proud .
2,"Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th...","▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁t...","▁in ▁school ▁, ▁we ▁spent ▁a ▁lot ▁of ▁time ▁s...","Ở trường , chúng tôi dành rất nhiều thời gian ...","In school , we spent a lot of time studying th..."
3,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...,▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ...,▁although ▁i ▁often ▁wondered ▁about ▁the ▁out...,Mặc dù tôi đã từng tự hỏi không biết thế giới ...,Although I often wondered about the outside wo...
4,"Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu...","▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁...","▁when ▁i ▁was ▁seven ▁years ▁old ▁, ▁i ▁saw ▁m...","Khi tôi lên 7 , tôi chứng kiến cảnh người ta x...","When I was seven years old , I saw my first pu..."
...,...,...,...,...,...,...
1263,"Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as...","▁tôi ▁thực ▁sự ▁tin ▁, ▁nếu ▁ta ▁coi ▁người ▁k...","▁i ▁truly ▁believe ▁, ▁if ▁we ▁can ▁see ▁one ▁...","Tôi thực sự tin , nếu ta coi người khác như nh...","I truly believe , if we can see one another as..."
1264,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...,▁những ▁tấm ▁hình ▁không ▁phải ▁là ▁về ▁bản ▁t...,▁these ▁images ▁are ▁not ▁of ▁issues ▁. ▁they ...,Những tấm hình không phải là về bản thân vấnđề...,These images are not of issues . They are of p...
1265,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...,▁không ▁có ▁ngày ▁nào ▁mà ▁tôi ▁không ▁nghĩ ▁v...,▁there ▁is ▁not ▁a ▁day ▁that ▁goes ▁by ▁that ...,Không có ngày nào mà tôi không nghĩ về những n...,There is not a day that goes by that I don &ap...
1266,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...,▁tôi ▁hi ▁vọng ▁những ▁tấm ▁hình ▁sẽ ▁đánh ▁th...,▁i ▁hope ▁that ▁these ▁images ▁awaken ▁a ▁forc...,Tôi hi vọng những tấm hình sẽ đánh thức một ng...,I hope that these images awaken a force in tho...


In [ ]:
data_vi = df_train["vi_token"].values.tolist()
data_en = df_train["en_token"].values.tolist()

with open("train.vi", "w") as f:
  f.write("\n".join(data_vi))

with open("train.en", "w") as f:
  f.write("\n".join(data_en))

In [ ]:
def remove_punc(text):
    return text.replace("▁", " ")

In [ ]:
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/training/clean-corpus-n.perl

--2024-06-22 11:18:25--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/training/clean-corpus-n.perl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4257 (4.2K) [text/plain]
Saving to: ‘clean-corpus-n.perl’

clean-corpus-n.perl 100%[===================>]   4.16K  --.-KB/s    in 0s      

2024-06-22 11:18:25 (52.7 MB/s) - ‘clean-corpus-n.perl’ saved [4257/4257]



In [ ]:
!perl clean-corpus-n.perl -max-word-length 50 train vi en train.clean 0 512

clean-corpus.perl: processing train.vi & .en to train.clean, cutoff 0-512, ratio 9
..........(100000)...
Input sentences: 133000  Output sentences:  132993


In [ ]:
test_vi = df_test["vi_token"].values.tolist()
test_en = df_test["en_token"].values.tolist()

with open("test.vi", "w") as f:
  f.write("\n".join(test_vi))

with open("test.en", "w") as f:
  f.write("\n".join(test_en))


test_vi = [i.replace("\n", "") for i in test_vi]
test_en = [i.replace("\n", "") for i in test_en]
test_vi[:10]

['▁khi ▁tôi ▁còn ▁nhỏ ▁, ▁tôi ▁nghĩ ▁rằng ▁bắc triều ▁tiên ▁là ▁đất ▁nước ▁tốt ▁nhất ▁trên ▁thế ▁giới ▁và ▁tôi ▁thường ▁hát ▁bài ▁& qu ot ; ▁chúng ▁ta ▁chẳng ▁có ▁gì ▁phải ▁ghen ▁tị ▁. ▁& qu ot ;',
 '▁tôi ▁đã ▁rất ▁tự ▁hào ▁về ▁đất ▁nước ▁tôi ▁.',
 '▁ở ▁trường ▁, ▁chúng ▁tôi ▁dành ▁rất ▁nhiều ▁thời ▁gian ▁để ▁học ▁về ▁cuộc ▁đời ▁của ▁chủ ▁tịch ▁kim ▁ii - ▁sung ▁, ▁nhưng ▁lại ▁không ▁học ▁nhiều ▁về ▁thế ▁giới ▁bên ▁ngoài ▁, ▁ngoại ▁trừ ▁việc ▁hoa ▁kỳ ▁, ▁hàn ▁quốc ▁và ▁nhật ▁bản ▁là ▁kẻ ▁thù ▁của ▁chúng ▁tôi ▁.',
 '▁mặc ▁dù ▁tôi ▁đã ▁từng ▁tự ▁hỏi ▁không ▁biết ▁thế ▁giới ▁bên ▁ngoài ▁kia ▁như ▁thế ▁nào ▁, ▁nhưng ▁tôi ▁vẫn ▁nghĩ ▁rằng ▁mình ▁sẽ ▁sống ▁cả ▁cuộc ▁đời ▁ở ▁bắc triều ▁tiên ▁, ▁cho ▁tới ▁khi ▁tất ▁cả ▁mọi ▁thứ ▁đột ▁nhiên ▁thay ▁đổi ▁.',
 '▁khi ▁tôi ▁lên ▁0 ▁, ▁tôi ▁chứng ▁kiến ▁cảnh ▁người ▁ta ▁xử ▁bắn ▁công ▁khai ▁lần ▁đầu ▁tiên ▁trong ▁đời ▁, ▁nhưng ▁tôi ▁vẫn ▁nghĩ ▁cuộc ▁sống ▁của ▁mình ▁ở ▁đây ▁là ▁hoàn ▁toàn ▁bình ▁thường ▁.',
 '▁gia ▁đình ▁của ▁tôi ▁không ▁nghèo ▁, ▁và 

In [ ]:
# Áp dụng hàm remove_punc và tách các từ
test_vi = [remove_punc(text).split() for text in test_vi]
test_en = [remove_punc(text).split() for text in test_en]


for i in range(10):
  print(test_en[i])
  print(test_vi[i])


['when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '&', 'quot', ';', 'nothing', 'to', 'envy', '.', '&', 'quot', ';']
['khi', 'tôi', 'còn', 'nhỏ', ',', 'tôi', 'nghĩ', 'rằng', 'bắc', 'triều', 'tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&', 'qu', 'ot', ';', 'chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&', 'qu', 'ot', ';']
['and', 'i', 'was', 'very', 'proud', '.']
['tôi', 'đã', 'rất', 'tự', 'hào', 'về', 'đất', 'nước', 'tôi', '.']
['in', 'school', ',', 'we', 'spent', 'a', 'lot', 'of', 'time', 'studying', 'the', 'history', 'of', 'kim', 'il', '-', 'sung', ',', 'but', 'we', 'never', 'learned', 'much', 'about', 'the', 'outside', 'world', ',', 'except', 'that', 'america', ',', 'south', 'korea', ',', 'japan', 'are', 'the', 'enemies', '.']
['ở', 'trường', ',', 'chúng', 'tôi', 'dành', 'rất', 'n

In [ ]:
!perl clean-corpus-n.perl -max-word-length 50 test vi en test.clean 0 512

clean-corpus.perl: processing test.vi & .en to test.clean, cutoff 0-512, ratio 9

Input sentences: 1268  Output sentences:  1268


In [ ]:
!wc -l train.clean.vi train.clean.en

  132993 train.clean.vi
  132993 train.clean.en
  265986 total


In [ ]:
!wc -l test.clean.vi test.clean.en

  1268 test.clean.vi
  1268 test.clean.en
  2536 total


In [ ]:
with open("train.clean.vi", "r") as f:
    train_vi = f.readlines()

with open("train.clean.en", "r") as f:
    train_en = f.readlines()

train_vi = [i.replace("\n", "") for i in train_vi]
train_en = [i.replace("\n", "") for i in train_en]
train_en[:10]

['▁rachel ▁pike : ▁the ▁science ▁behind ▁a ▁climate ▁headline',
 '▁in ▁0 ▁minutes ▁, ▁atmospheric ▁chemist ▁rachel ▁pike ▁provides ▁a ▁glimpse ▁of ▁the ▁massive ▁scientific ▁effort ▁behind ▁the ▁bold ▁headlines ▁on ▁climate ▁change ▁, ▁with ▁her ▁team ▁-- ▁one ▁of ▁thousands ▁who ▁contributed ▁-- ▁taking ▁a ▁risky ▁flight ▁over ▁the ▁rainforest ▁in ▁pursuit ▁of ▁data ▁on ▁a ▁key ▁molecule ▁.',
 '▁i ▁& ap os ; d ▁like ▁to ▁talk ▁to ▁you ▁today ▁about ▁the ▁scale ▁of ▁the ▁scientific ▁effort ▁that ▁goes ▁into ▁making ▁the ▁headlines ▁you ▁see ▁in ▁the ▁paper ▁.',
 '▁headlines ▁that ▁look ▁like ▁this ▁when ▁they ▁have ▁to ▁do ▁with ▁climate ▁change ▁, ▁and ▁headlines ▁that ▁look ▁like ▁this ▁when ▁they ▁have ▁to ▁do ▁with ▁air ▁quality ▁or ▁sm og ▁.',
 '▁they ▁are ▁both ▁two ▁branches ▁of ▁the ▁same ▁field ▁of ▁atmospheric ▁science ▁.',
 '▁recently ▁the ▁headlines ▁looked ▁like ▁this ▁when ▁the ▁inter governmental ▁panel ▁on ▁climate ▁change ▁, ▁or ▁ip cc ▁, ▁put ▁out ▁their ▁report ▁on ▁

In [ ]:
# Áp dụng hàm remove_punc và tách các từ
train_vi = [remove_punc(text).split() for text in train_vi]
train_en = [remove_punc(text).split() for text in train_en]


for i in range(10):
  print(train_en[i])
  print(train_vi[i])


['rachel', 'pike', ':', 'the', 'science', 'behind', 'a', 'climate', 'headline']
['khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']
['in', '0', 'minutes', ',', 'atmospheric', 'chemist', 'rachel', 'pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.']
['trong', '0', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'rachel', 'pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng', 'sau', 'những', 'tiêu', 'đề', 'táo', 'bạo', 'về', 'biến', 'đổi', 'khí', 'hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên', 'cứu', 'của', 'mình', '--', 'hàng', 'ngàn', 'người', 'đã', 'cống', 'hiến', 'cho', 'dự', 'án', 'này', '--', 'một

## Make vocab

In [ ]:
def make_vocab(train_data, min_freq):
  vocab = {}
  for tokenlist in train_data:
    for token in tokenlist:
      if token not in vocab:
        vocab[token] = 0
      vocab[token] += 1 # Đếm các từ data
  vocablist = [('<unk>',0),('<pad>', 0), ('<cls>', 0), ('<eos>', 0)] # Danh sách các tuple đặc trưng trước
  vocabidx = {} # dict của tuple
  for token, freq in vocab.items(): # Từ đó , với số lượng của từ đó trong train data
    if freq >= min_freq: # Số lượng từ mà > 3
      idx = len(vocablist) # đang là 4
      vocablist.append((token, freq)) #Từ đó , với số lượng của từ đó trong train data
      vocabidx[token] = idx  # Từ điển (dict)
  vocabidx['<unk>'] = 0
  vocabidx['<pad>'] = 1
  vocabidx['<cls>'] = 2
  vocabidx['<eos>'] = 3
  return vocablist, vocabidx

vocablist_en, vocabidx_en = make_vocab(train_en, 3)
vocablist_vi, vocabidx_vi = make_vocab(train_vi, 3)

print('vocab size en: ', len(vocablist_en))
print('vocab size vi: ', len(vocablist_vi))

vocab size en:  20317
vocab size vi:  8879


## Preprocess

In [ ]:
def preprocess(data, vocabidx):
  rr = []
  for tokenlist in data:
    tkl =['<cls>']
    for token in tokenlist:
      tkl.append(token if token in vocabidx else '<unk>')
    tkl.append('<eos>')
    rr.append((tkl))
  return rr

train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)
for i in range(5):
  print(train_en_prep[i])
  print(train_vi_prep[i])
  print(test_en_prep[i])

['<cls>', 'rachel', 'pike', ':', 'the', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
['<cls>', 'khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu', '<eos>']
['<cls>', 'when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '&', 'quot', ';', 'nothing', 'to', 'envy', '.', '&', 'quot', ';', '<eos>']
['<cls>', 'in', '0', 'minutes', ',', 'atmospheric', 'chemist', 'rachel', 'pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
['<cls>', 'trong', '0', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'rachel', 'pike',

In [ ]:
train_data = list(zip(train_en_prep, train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))

test_data = list(zip(test_en_prep, test_en, test_vi))

for i in range(10):
  print(train_data[i])
for i in range(10):
  print(test_data[i])

(['<cls>', '.', '<eos>'], ['<cls>', '.', '<eos>'])
(['<cls>', '.', '<eos>'], ['<cls>', '.', '<eos>'])
(['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'ok', '<eos>'])
(['<cls>', 'oh', '.', '<eos>'], ['<cls>', 'ồ', '<eos>'])
(['<cls>', 'seven', '.', '<eos>'], ['<cls>', 'bảy', '<eos>'])
(['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'ừ', '<eos>'])
(['<cls>', 'kant', '.', '<eos>'], ['<cls>', 'kant', '<eos>'])
(['<cls>', 'mill', '.', '<eos>'], ['<cls>', 'mill', '<eos>'])
(['<cls>', '0,000', '.', '<eos>'], ['<cls>', '0000', '<eos>'])
(['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'ok', '<eos>'])
(['<cls>', 'when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singing', 'a', 'song', 'called', '&', 'quot', ';', 'nothing', 'to', 'envy', '.', '&', 'quot', ';', '<eos>'], ['when', 'i', 'was', 'little', ',', 'i', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'i', 'grew', 'up', 'singin

##Make Batch

In [ ]:
def make_batch(data, batchsize):
  bb = []
  ben = []
  bvi = []
  for en,vi in data:
    ben.append(en)
    bvi.append(vi)
    if len(ben) >= batchsize:
      bb.append((ben, bvi))
      ben = []
      bvi = []
  if len(ben) > 0:
    bb.append((ben, bvi))
  return bb

train_data = make_batch(train_data, BATCHSIZE)

for i in range(2):
  print(train_data[i])

([['<cls>', '.', '<eos>'], ['<cls>', '.', '<eos>'], ['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'oh', '.', '<eos>'], ['<cls>', 'seven', '.', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'kant', '.', '<eos>'], ['<cls>', 'mill', '.', '<eos>'], ['<cls>', '0,000', '.', '<eos>'], ['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'um', '.', '<eos>'], ['<cls>', 'wonderful', '.', '<eos>'], ['<cls>', 'good', '.', '<eos>'], ['<cls>', 'no', '.', '<eos>'], ['<cls>', 'mario', '!', '<eos>'], ['<cls>', 'books', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'good', '.', '<eos>'], ['<cls>', 'so', '...', '<eos>'], ['<cls>', 'yes', '.', '<eos>'], ['<cls>', 'so', '...', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'useless', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'joey', '.', '<eos>'], ['<cls>', 'joey', '.', '<eos>'], ['<cls>', 'germany', '.', '<eos>'], ['<cls>', 'why', '?', '<eos>'], ['<cls>', 'negative', '.

##Padding Batch

In [ ]:
def padding_batch(b):
  maxlen = max([len(x) for x in b])
  for tkl in b:
    for i in range(maxlen - len(tkl)):
      tkl.append('<pad>')

def padding(bb):
  for ben, bvi in bb:
    padding_batch(ben)
    padding_batch(bvi)

padding(train_data)

for i in range(3):
  print(train_data[i])

([['<cls>', '.', '<eos>', '<pad>'], ['<cls>', '.', '<eos>', '<pad>'], ['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'oh', '.', '<eos>'], ['<cls>', 'seven', '.', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'kant', '.', '<eos>'], ['<cls>', 'mill', '.', '<eos>'], ['<cls>', '0,000', '.', '<eos>'], ['<cls>', 'ok', '.', '<eos>'], ['<cls>', 'um', '.', '<eos>'], ['<cls>', 'wonderful', '.', '<eos>'], ['<cls>', 'good', '.', '<eos>'], ['<cls>', 'no', '.', '<eos>'], ['<cls>', 'mario', '!', '<eos>'], ['<cls>', 'books', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'good', '.', '<eos>'], ['<cls>', 'so', '...', '<eos>'], ['<cls>', 'yes', '.', '<eos>'], ['<cls>', 'so', '...', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'useless', '.', '<eos>'], ['<cls>', 'hey', '.', '<eos>'], ['<cls>', 'yeah', '.', '<eos>'], ['<cls>', 'joey', '.', '<eos>'], ['<cls>', 'joey', '.', '<eos>'], ['<cls>', 'germany', '.', '<eos>'], ['<cls>', 'why', '?', '<eos>'], ['<cls

In [ ]:
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi[token] for token in tokenlist] for tokenlist in bvi]) for ben, bvi in train_data]
test_data = [([vocabidx_en[token] for token in enprep], en, vi) for enprep, en, vi in test_data]
for i in range(3):
  print(train_data[i])
  print(test_data[i])

([[2, 46, 3, 1], [2, 46, 3, 1], [2, 1122, 46, 3], [2, 1630, 46, 3], [2, 2010, 46, 3], [2, 2463, 46, 3], [2, 15272, 46, 3], [2, 3061, 46, 3], [2, 959, 46, 3], [2, 1122, 46, 3], [2, 7150, 46, 3], [2, 1011, 46, 3], [2, 1279, 46, 3], [2, 512, 46, 3], [2, 1655, 1751, 3], [2, 2047, 46, 3], [2, 4184, 46, 3], [2, 4184, 46, 3], [2, 1279, 46, 3], [2, 211, 1885, 3], [2, 2308, 46, 3], [2, 211, 1885, 3], [2, 2463, 46, 3], [2, 13828, 46, 3], [2, 4184, 46, 3], [2, 2463, 46, 3], [2, 16039, 46, 3], [2, 16039, 46, 3], [2, 1453, 46, 3], [2, 863, 554, 3], [2, 1385, 46, 3], [2, 991, 46, 3]], [[2, 71, 3, 1], [2, 71, 3, 1], [2, 1263, 3, 1], [2, 1406, 3, 1], [2, 2402, 3, 1], [2, 1828, 3, 1], [2, 6420, 3, 1], [2, 5685, 3, 1], [2, 140, 3, 1], [2, 1263, 3, 1], [2, 6057, 3, 1], [2, 894, 3, 1], [2, 1060, 3, 1], [2, 100, 3, 1], [2, 1308, 1354, 3], [2, 408, 71, 3], [2, 54, 71, 3], [2, 54, 71, 3], [2, 1060, 71, 3], [2, 83, 44, 3], [2, 90, 71, 3], [2, 451, 1414, 3], [2, 4873, 71, 3], [2, 206, 423, 3], [2, 54, 71, 3], 

#2. Prepare For Model

In [ ]:
from torch import Tensor
from torch import nn
from torch.nn import Transformer
import math

## Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 900):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


## Token Embedding

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

## Seq2Seq Transformer

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.01):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

#3. Model Transformer

In [ ]:
import time
torch.manual_seed(0)

MODELNAME = 'transfomers.model'
SRC_VOCAB_SIZE = len(vocablist_en)
TGT_VOCAB_SIZE = len(vocablist_vi)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
model = model.to(DEVICE)
PAD_IDX= 1
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == vocabidx_en['<pad>']).transpose(0, 1)
    tgt_padding_mask = (tgt == vocabidx_vi['<pad>']).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

#4. Train Model

In [ ]:
def train():
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
  model.train()
  all_time= 0
  for epoch in range(15):
    start= time.time()
    loss = 0

    for en, vi in train_data:

      en = torch.tensor(en, dtype=torch.int64).transpose(0,1).to(DEVICE)
      vi = torch.tensor(vi, dtype=torch.int64).transpose(0,1).to(DEVICE)
      tgt_input = vi[:-1, :]


      optimizer.zero_grad()
      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(en, tgt_input)
      y = model(en, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
      tgt_out = vi[1:, :]
      batchloss = loss_fn(y.reshape(-1, y.shape[-1]), tgt_out.reshape(-1))
      batchloss.backward()
      optimizer.step()
      loss = loss + batchloss.item()
    end= time.time()
    all_time+=(end-start)
    print("epoch", epoch, ": loss", loss, "Epoch time",end-start)
  torch.save(model.state_dict(), MODELNAME)

In [ ]:
!pip install prettytable
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+-------------------------------------------------------------+------------+
|                           Modules                           | Parameters |
+-------------------------------------------------------------+------------+
|    transformer.encoder.layers.0.self_attn.in_proj_weight    |   786432   |
|     transformer.encoder.layers.0.self_attn.in_proj_bias     |    1536    |
|    transformer.encoder.layers.0.self_attn.out_proj.weight   |   262144   |
|     transformer.encoder.layers.0.self_attn.out_proj.bias    |    512     |
|         transformer.encoder.layers.0.linear1.weight         |   262144   |
|          transformer.encoder.layers.0.linear1.bias          |    512     |
|         transformer.encoder.layers.0.linear2.weight         |   262144   |
|          transformer.encoder.layers.0.linear2.bias          |    512     |
|          transformer.encoder.layers.0.norm1.weight          |    512     |
|           transformer.encoder.layers.0.norm1.bias           |    512     |

32128175

In [ ]:
train()

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


epoch 0 : loss 16484.20712852478 Epoch time 210.6608588695526
epoch 1 : loss 11130.670394480228 Epoch time 201.0581738948822
epoch 2 : loss 9336.488625824451 Epoch time 200.2773859500885
epoch 3 : loss 8279.163113236427 Epoch time 200.22485733032227
epoch 4 : loss 7530.962986648083 Epoch time 200.48572778701782
epoch 5 : loss 6936.537015080452 Epoch time 200.13974404335022
epoch 6 : loss 6439.348349332809 Epoch time 206.57535529136658
epoch 7 : loss 5999.309655278921 Epoch time 201.06469583511353
epoch 8 : loss 5611.371713489294 Epoch time 201.1739068031311
epoch 9 : loss 5263.602762192488 Epoch time 201.48279762268066
epoch 10 : loss 4947.156174957752 Epoch time 200.87677240371704
epoch 11 : loss 4660.9475983679295 Epoch time 200.88284969329834
epoch 12 : loss 4395.115301281214 Epoch time 201.697895526886
epoch 13 : loss 4153.152097672224 Epoch time 201.20533871650696
epoch 14 : loss 3931.7374463379383 Epoch time 201.5876784324646


In [ ]:
len(train_data)

4157

In [ ]:
len(test_data)

1268

#5. Test Model

In [ ]:
import torchtext.data.metrics
def evaluate(model, src, max_len, start_symbol):
    num_tokens = src.shape[0]
    src_mask = torch.zeros((num_tokens, num_tokens),device=DEVICE).type(torch.bool)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    pred = []
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        # next_word = out.squeeze().argmax()
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        # print(prob)
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == vocabidx_en['<eos>']:
            break
        pred_y = vocablist_vi[next_word][0]
        pred.append(pred_y)
    return pred

def test():
  model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)
  model.load_state_dict(torch.load(MODELNAME))
  model.eval()
  ref = []
  pred = []
  for enprep, en, vi in test_data:
    input = torch.tensor([enprep], dtype=torch.int64).transpose(0,1).to(DEVICE).view(-1, 1)
    p = evaluate(model, input, 50, vocabidx_en['<cls>'])
    ref.append([vi])
    pred.append(p)
  bleu = torchtext.data.metrics.bleu_score(pred, ref)
  print("total: ", len(test_data))
  print("bleu: ", bleu)

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
test()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


total:  1268
bleu:  0.17614158987998962


In [ ]:
model.eval()

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.01, inplace=False)
          (linear2): Linear(in_features=512, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.01, inplace=False)
          (dropout2): Dropout(p=0.01, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
          (self_attn): MultiheadAtt